In [1]:
from nanotag.data import NanotagData, ImageFileCollection, Summary
from nanotag.image import ImageSeries, GaussianFilterSlider
from nanotag.tags import PointTags, PointTagSeries
from nanotag.timeline import Timeline, TimelineTags, EditTimelineTags
from nanotag.canvas import Canvas, ToolBox
from nanotag.tools import EditPointTags, ResetView, BoxZoomTool, PanZoomTool
from nanotag.utils import link, redirected_path
from nanotag.histogram import Histogram
from traitlets import directional_link
import json
from bqplot import ColorScale, Scatter, Figure
import ipywidgets as widgets
import os
from pathlib import Path
import numpy as np
from nanotag.graph.stable_delaunay_graph import stable_delaunay_faces
from nanotag.graph.representation import faces_to_edges
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import single_source_shortest_path



from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

def parse_metadata(metadata):
    metadata = metadata['ImageDescription']
    metadata = metadata[metadata.find('{'):]
    metadata = json.loads(metadata)
    return metadata

def parse_voa_current(metadata):
    try:
        timeseries = parse_metadata(metadata)['properties']['timeseries'] 
        return [t['VOA_current'] for t in timeseries]
    except KeyError:
        return 'Not found'

In [3]:
root_directory = '/data'
filt = '**/*.tif'

#root_directory = 'D:/data/hBN_export'
#filt = '40_50kV/**/*.tif'


image_series = ImageSeries()
image_file_collection = ImageFileCollection(filter=filt, image_series=image_series)
image_file_collection.root_directory = root_directory

canvas = Canvas()
gaussian_filter = GaussianFilterSlider(max=10)
image_series.filters = [gaussian_filter]
canvas.images = [image_series]

has_defect_tags = TimelineTags(row='defect', data_fields=('has_defect',), enable_move=False, color_scale=ColorScale(min=0, max=1))
area_tags = TimelineTags(row='lattice area', data_fields=('lattice area',), enable_move=False, color_scale=ColorScale(colors=['red', 'white'], min=.8, max=1))

timeline = Timeline()
timeline.tags = [has_defect_tags, area_tags]
directional_link((timeline, 'frame_index'), (image_series, 'frame_index'))
link((image_series, 'num_frames'), (timeline, 'num_frames'))

atom_tags = PointTagSeries(data_fields=('defect','sublattice'), enable_move=False)

directional_link((timeline, 'frame_index'), (atom_tags, 'frame_index'))
canvas.tags = [atom_tags]

toolbox = ToolBox(canvas)
toolbox.tools = {'Reset' : ResetView(),
                 'BoxZoom' : BoxZoomTool(),
                 'Pan' : PanZoomTool(),
                 'B defect' : EditPointTags(atom_tags, data_fields={'defect':1, 'sublattice':0}),
                 'N defect' : EditPointTags(atom_tags, data_fields={'defect':1, 'sublattice':1}),
                }

nanotag_data = NanotagData(root_directory = root_directory,
                           read_file = '/analysis/50_50kV/40_01.json',
                           tags = {'points' : atom_tags, 'area': area_tags}
                           )

update_button = widgets.Button(description='Update')

summary = Summary(write_file='summary.json')

atom_tags.point_tags.color_scheme = 'category'
atom_tags.point_tags.color_scale.min = 0
atom_tags.point_tags.color_scale.max = 10
toolbox.button_width = '87px'

def transform_summary_path(x):
    x = redirected_path(nanotag_data.root_directory, x, 'summaries')
    x = os.path.splitext(x)[0] + '.json'
    x = str(Path(x))
    os.makedirs(os.path.split(x)[0], exist_ok=True)
    return x

def transform_analysis_path(x):
    x = redirected_path(nanotag_data.root_directory, x, 'final_analysis')
    x = os.path.splitext(x)[0] + '.json'
    x = str(Path(x))
    os.makedirs(os.path.split(x)[0], exist_ok=True)
    return x

directional_link((image_file_collection, 'path'), (summary, 'write_file'), transform_summary_path)
directional_link((image_file_collection, 'path'), (nanotag_data, 'read_file'), transform_analysis_path)

def update_defects(*args):
    has_defect = np.where([np.any(np.array(x['defect']) == 1) for x in atom_tags.series.values()])[0]
    has_defect_tags.t = has_defect

def get_first_defect_frame(*args):
    defect_frames = has_defect_tags.t
    
    if len(defect_frames) > 0:
        return int(min(defect_frames))
    else:
        return None

def rebuild_summary(*args):
    summary_data = {}
    summary_data['path'] = image_file_collection.relative_path
    summary_data['num_frames'] = image_series.num_frames
    summary_data['shape_x'] = image_series.images.shape[1]
    summary_data['shape_y'] = image_series.images.shape[2]
    
    if 'sampling' in nanotag_data.data:
        summary_data['sampling'] = nanotag_data.data['sampling']
        summary_data['fov'] = image_series.images.shape[1] * image_series.images.shape[2] * nanotag_data.data['sampling']
    else:
        summary_data['sampling'] = None
        summary_data['fov'] = None
    
    i = get_first_defect_frame()
    
    if i is None:
        summary_data['first_defect_frame'] = None
        summary_data['first_defect_x'] = None
        summary_data['first_defect_y'] = None
        summary_data['first_defect_type'] = None
        first_defect = None
    else:
        is_defect = np.array(atom_tags.series[i]['defect']) == 1
        first_defect = np.argmin(np.array(atom_tags.series[i]['y'])[is_defect])
        
        points = np.array([atom_tags.series[i]['x'], atom_tags.series[i]['y']]).T
        faces = stable_delaunay_faces(points, 2)
        edges = faces_to_edges(faces)
        G = nx.Graph()
        G.add_edges_from(edges)
        
        targets = np.where(is_defect)[0]
        first_defect = np.argmin(np.array(atom_tags.series[i]['y'])[is_defect])

        source = targets[first_defect]
        paths = single_source_shortest_path(G, source)
        path_lengths = [len(paths[target]) for target in targets][1:]
        
        if len(path_lengths) > 0:
            min_path_length = min(path_lengths)
        else:
            min_path_length = np.inf
        
        summary_data['first_defect_frame'] = i
        summary_data['first_defect_x'] = np.array(atom_tags.series[i]['x'])[is_defect][first_defect]
        summary_data['first_defect_y'] = np.array(atom_tags.series[i]['y'])[is_defect][first_defect]
        
        if min_path_length <= 3:
            summary_data['first_defect_type'] = 'unknown'
        else:
            summary_data['first_defect_type'] = {0:'B', 1:'N'}[np.array(atom_tags.series[i]['sublattice'])[is_defect][first_defect]]
    
    if i is None:
        summary_data['voa'] = parse_voa_current(image_file_collection.metadata)
    else:
        summary_data['voa'] = parse_voa_current(image_file_collection.metadata)[:i + 1]
    
    
    summary.current_data = summary_data

def jump_to_first_frame():
    i = get_first_defect_frame()
    
    if i is None:
        i = image_series.num_frames - 1
        
    timeline.frame_index = i
    
def on_new_data(*args):
    update_defects()
    rebuild_summary()
    jump_to_first_frame()

def update(*args):
    update_defects()
    rebuild_summary()
    jump_to_first_frame()

atom_tags.observe(on_new_data, 'series')
update_button.on_click(update)

from nanotag.events import KeyEvents

def toggle_point_visible(*args):
    atom_tags.point_tags.visible = not atom_tags.point_tags.visible


In [4]:
nanotag_data.read_data()
canvas.reset()
on_new_data()
rebuild_summary()

update_row = widgets.HBox([update_button])
tools_column = widgets.VBox([image_file_collection, nanotag_data, gaussian_filter, atom_tags, toolbox, update_row])

canvas_box = widgets.VBox([canvas, timeline])


app = widgets.HBox([canvas_box, tools_column, summary])

events = KeyEvents(canvas_box)

events.callbacks = {'ArrowLeft' : timeline.previous_frame,
                    'ArrowRight' : timeline.next_frame,
                    'ArrowUp' : image_file_collection.next_file,
                    'ArrowDown' : image_file_collection.previous_file,
                    'r' : canvas.reset,
                    'z' : lambda : toolbox.toggle_tool('BoxZoom'),
                    'p' : lambda : toolbox.toggle_tool('Pan'),
                    'v' : toggle_point_visible,
                    'b' : lambda : toolbox.toggle_tool('B defect'),
                    'n' : lambda : toolbox.toggle_tool('N defect'),
                   }

app